## Indentation
- *indentation* means shifting a line of code by either a given number of spaces or a tab (`Tab` key);
- a tab is a *single* special character that is visualised as an empty space;
- tab-style indentation may have been popular in the past, but today the standard is space-style indentation using 4 whitespaces;
- most editors will produce 4 whitespaces by default (or can be set up to do so!)

### In python
- indentation in python is ***part of the syntax!***
- indentation delimits the code of a function, an `if/elif/else` clause, a loop etc.
- any number of spaces is recognised, but it has to be consistent



## Dictionaries tips and tricks

In [15]:
a_dict = { letter: number for number, letter in enumerate('abcdef') }

print(a_dict)


{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5}


In [12]:
# print(a_dict['h']) # not nice

In [18]:
print(a_dict.get('h'))

None


In [25]:
print(a_dict.get('f',-1))
print(a_dict.get('g',-1))

5
-1


## `defaultdict`
`defaultdict` is a dictionary associated to a function, the output of this function is the default value for all non-existent keys

In [11]:
from collections import defaultdict

In [15]:
def init_value():
    return "default value"

a = defaultdict(init_value)
a[0] = "custom value"
print(a[0])
print(a[1])

print(a)

custom value
default value
defaultdict(<function init_value at 0x7f1d504b7c70>, {0: 'custom value', 1: 'default value'})


More useful when we do not know all our keys in advance.

In [16]:
freq = defaultdict(int)

message = "Hello World!"

for letter in message:
    freq[letter] += 1
 
print(freq)

defaultdict(<class 'int'>, {'H': 1, 'e': 1, 'l': 3, 'o': 2, ' ': 1, 'W': 1, 'r': 1, 'd': 1, '!': 1})


# Functions

What are functions?
- from maths: relation between two sets A and B that assign to each element of A exactly one element of B
- in computing: similar concept, but in general (1) A and B are loosely defined (2) the association is determined through an algorithm.

## Example function

In [1]:
def f(a, b):
    c = a + b
    return c

In [2]:
f(3,2)

5

- `a` and `b` are called *arguments* and represent the input of the function;
- `c` is called *return value* and represent the output of the function;
- the combination of arguments and return values takes the name of *signature* of a function (not very important in `python`, much more in compiled languages);
- functions return **one** item, don't they?

In [3]:
def g(a, b):
    c = a + b
    d = a - b
    return c, d

def h():
    return
    # pass

In [4]:
sum, diff = g(3,2)
print(sum, diff)

5 1


In [5]:
print(g(3,2), type(g(3,2)))

(5, 1) <class 'tuple'>


In [6]:
print(h(), type(h()))

None <class 'NoneType'>


Functions in `python` return always exactly one item, but this item can actually be a collection. If you try to return more than one value, python will create a tuple for you.

## Positional arguments and keyword arguments
- when calling a function with a sequence of arguments, we say we are *passing* the arguments to the function;
- `python` function accepts arguments passing either by position or by keyword;
- keyword arguments must alwasy follow positional arguments!

In [30]:
def f(a, b, c):
    print(a, b, c)

f(1,2,3) # position
f(a=1, b=2, c=3) # keywords
f(1, 2, c=3) # mixed
# f(a=1, 2, 3)

1 2 3
1 2 3
1 2 3


### Do all arguments need to be defined a priori?
**No**!

In [47]:
def f(arg, *args, **kwargs):
    print(arg) # a name
    print(args) # a tuple
    print(kwargs) # a dictionary

In [48]:
f(1,2,3,d=4, e=5)

1
(2, 3)
{'d': 4, 'e': 5}


This gives you a lot of flexibility but be careful in taking advantage of it. It is not a good idea to have too little control on what is passed to your function.

### What if I want more control?
You can separate positional arguments from keyword arguments with a `*`. All the arguments after the `*` will be required to be passed as keywords.

In [1]:
def f(a, b, *, c, d):
    return a * d - b * c

In [3]:
# f(1,2,3,4)
f(1,2, c=3, d=4)

-2

## Type hints
- python is dynamically typed: you can do whatever you want and there will be little control about the types you use!
- from python 3.5 *type hints* are supported: we can indicate what types a function is supposed to take as arguments and what type it returns!

In [7]:
def f(a : int, b : int) -> str:
    if a > b:
        return f"{a} is greater than {b}"
    else:
        return f"{a} is less than or equal to {b}"

print(f(1, 2))

print(f(1.5, 3.5))

1 is less than or equal to 2
1.5 is less than or equal to 3.5


- the python interpreter does not complain if you don't respect type hints, after all it is a *dynamically typed* language!
- type hints are useful **to you** to remember how a function is supposed to behave: they may seem (and probably are) unnecessary at this level but it is important to **pick good habits** from the start! 
- there are tools known as **static type checkers** (one is `mypy`) that can check if your code respect all the type declarations.

In [9]:
# one more advanced example...

from typing import Collection

def sum_list(v : Collection[float]) -> float:
    return sum(v)

sum_list([1,5,6])

12

## Composition
Naturally, a function can call another function.

In [57]:
def f(x):
    return x + 1

def inv_f(x):
    return x - 1

f(inv_f(10))

10

## Recursion
- Recursion is a classic paradigm of computing where a function includes a call to itself in its body.
- For recursion to work without resulting in an infinite loop eating all your computer's memory, there must be conditions that cause the function to exit without calling itself.
- Most naturally recursive problems can naturally be resolved with iterations.

In [19]:
def fibonacci(n : int) -> int:
    """
    Calculate the n-th fibonacci number
    """
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)


In [20]:
fibonacci(4)

3

## Lambda function
The `lambda` statement in python allows to define anonymous functions.



In [38]:
sq1 = lambda a : a**2

def sq2(x):
    return x ** 2

In [39]:
print(sq1(5))
print(sq2(5))


25
25


Like this it seems just a redundant syntax, however it allows more powerful uses like generating functions parametrically.

In [41]:
def pow(n):
    return lambda x : x**n

sq = pow(2)
cu = pow(3)

In [48]:
print(sq(2), cu(2))

4 8


Another common paradigm: let's say you have a function `f(x, y)` that you want to use many times fixing `x` and varying `y`.

In [46]:
def times(x, y):
    return x * y

x0 = 5
times_5 = lambda a : times(x0, a)

In [47]:
print(g(5)) 

25


## Built-in functions
A list of built-in functions is available at https://docs.python.org/3.11/library/functions.
- numeric: `abs()` absolute value, `divmod()`  combination of `//` and `%`, `min()`, `max()`, `sum()`, `pow()`, `round()`,
- boolean: `any()`, `all()`
- list manipulation: `reversed()`, `sorted()`
- character conversion: `chr()` (int to character)


### Map and filter...

In [53]:
a_list = [1, 2, 3, "a", "b", "c"]

list_numbers = filter(lambda x: isinstance(x, int), a_list)
list_letters = filter(lambda x: isinstance(x, str), a_list)

print(list_numbers)

for n in list_numbers:
    print(n)

1
2
3


What is happening here? The filter object is an iterator and needs to be "unwrapped".

In [54]:
n_list = range(10)

square_list = map(lambda x : x**2, n_list)

print(square_list)

In [55]:
print(list(square_list))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


In [59]:
%%timeit

square_list = map(lambda x : x**2, n_list)


135 ns ± 3.51 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [61]:
%%timeit
square_list = [x**2 for x in n_list]

2.29 µs ± 12.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
